
# 3D field ROM example for field snapshot input and output as Numpy array

This example shows how the example given in `ref_example_TBROM_inputField`
can be modified to take inputs as Numpy arrays, rather than reading from disk.
Similarly, outputs are produced as Numpy arrays.

The results arrays will be unflattened and results combined into a DataFrame of
x, y and z location and corresponding x, y and z displacement components.

Results will be exported for the whole model and a named selection consisting
only of the bolts.


<img src="file://_static/TBROM_input_field.png" width="400pt" align="center">



In [ ]:
# sphinx_gallery_thumbnail_path = '_static/TBROM_input_field.png'

<div class="alert alert-info"><h4>Note</h4><p>To be able to use the functionalities to generate an output field snapshot
  on demand, you must have a twin with one or more TBROMs.

  To project an input field snapshot, one or more of the TBROMs must be
  parameterized by input field data.

  To be able to use the functionalities to generate points file on demand for
  a TBROM, the geometry must have been embedded when exporting the TBROMs to
  Twin Builder, prior to building the twin.

  The twin inputs and outputs must follow specific naming conventions,
  summarised here and described in detail in
  `ref_example_TBROM_inputField`.

  - If there are multiple TBROMs in the twin, the format for the name of the
    twin input must be ``{input_field_name}_mode_{mode_index}_{tbrom_name}``
    and the output must be ``outField_mode_{mode_index}_{tbrom_name}``.
  - If there is a single TBROM in the twin, the format for the name of the
    twin input must be ``{input_field_name}_mode_{mode_index}`` and the output
    must be ``outField_mode_{mode_index}``.</p></div>




## Perform required imports
Perform required imports, which include downloading and importing the input
files.



In [ ]:
import numpy as np
import pandas as pd
from pytwin import TwinModel, download_file, read_binary

twin_file = download_file("ThermalTBROM_FieldInput_23R1.twin", "twin_files", force_download=True)
inputfieldsnapshot = download_file("TEMP_1.bin", "twin_input_files/inputFieldSnapshots", force_download=True)

## Define auxiliary functions
Define auxiliary function to unflatten a vector.



In [ ]:
def unflatten_vector(vector: np.ndarray, dimensionality: int):
    """Unflatten a vector to array with specified number of columns."""
    return vector.reshape(-1, dimensionality)

## Define ROM inputs
Define the twin scalar input pressure as 4 MPa.



In [ ]:
scalar_input = {"Pressure_Magnitude": 4000000.0}

Read one vector input from a file (this input could have been generated by a
separate program).



In [ ]:
temperature_array = read_binary(inputfieldsnapshot)

Enter information regarding TBROM, input fields and output named selection
(see `ref_example_TBROM_inputField` for examples of setting this
programmatically).



In [ ]:
romname = "test23R1_1"
fieldname = "inputTemperature"
out_ns = "Group_1"
input_name = "Pressure_Magnitude"
field_input = {romname: {fieldname: temperature_array}}

## Load the twin runtime and generate displacement results
Load the twin runtime and generate temperature results from the TBROM.



In [ ]:
print("Loading model: {}".format(twin_file))
twin_model = TwinModel(twin_file)

### Evaluate the twin with one set of input values and collect corresponding outputs
Because the twin is based on a static model, results for any inputs can be
obtained by setting the initial input values to the desired values and running
the initialization function.



In [ ]:
# initialize twin with input values
twin_model.initialize_evaluation(inputs=scalar_input, field_inputs=field_input)
# generate the field output array on the entire domain
outfield = twin_model.generate_snapshot(romname, on_disk=False)
# generate the field output on "Group_1"
outfield_bolts = twin_model.generate_snapshot(romname, on_disk=False, named_selection=out_ns)
# generate the points location array for the whole domain
points = twin_model.generate_points(romname, on_disk=False)
# generate the points location array on "Group_1"
points_bolts = twin_model.generate_points(romname, on_disk=False, named_selection=out_ns)

Unflatten vectors and combine into DataFrame. Both point location and
displacement have three components when unflattening.



In [ ]:
disp_xyz = unflatten_vector(outfield, 3)
loc_xyz = unflatten_vector(points, 3)
results = pd.DataFrame(
    {
        "x": loc_xyz[:, 0],
        "y": loc_xyz[:, 1],
        "z": loc_xyz[:, 2],
        "ux": disp_xyz[:, 0],
        "uy": disp_xyz[:, 1],
        "uz": disp_xyz[:, 2],
    }
)
print(f"Full results table:\n{results}")

Repeat for the subset of results on the bolts only



In [ ]:
disp_xyz = unflatten_vector(outfield_bolts, 3)
loc_xyz = unflatten_vector(points_bolts, 3)
results_bolts = pd.DataFrame(
    {
        "x": loc_xyz[:, 0],
        "y": loc_xyz[:, 1],
        "z": loc_xyz[:, 2],
        "ux": disp_xyz[:, 0],
        "uy": disp_xyz[:, 1],
        "uz": disp_xyz[:, 2],
    }
)
print(f"Bolt results table:\n{results_bolts}")